In [2]:
import os
from osgeo import gdal, ogr, osr
import pyproj
import numpy as np
import pandas as pd
import json

gdal.UseExceptions()

In [3]:
os.environ['PROJ_DATA']='/workspace/.conda/envs/env_labels/share/proj' 
# os.environ['GDAL_DATA']='/workspace/.conda/envs/env_labels/share/gdal'
# os.environ['GTIFF_SRS_SOURCE'] = 'EPSG'

In [4]:
import pystac
from pystac import Link, Asset
from pystac.extensions.label import LabelExtension
from pystac.extensions.label import LabelType
from pystac.extensions.label import LabelClasses
from pystac.extensions.label import LabelStatistics
from pystac.extensions.version import ItemVersionExtension

In [5]:
# Set dataframe to None 
df = None

In [6]:
# Read GeoJSON file and extract point coordinates
def read_geojson_coordinates(geojson_file):
    with open(geojson_file, 'r') as file:
        geojson_data = json.load(file)
    #for f in geojson_data['features'][:10]: print(f)
    
    points = []
    luc = []
    for feature in geojson_data['features']:
        if feature['geometry']['type'] == 'Point':
            # Add lon and lat
            lon, lat, _ = feature['geometry']['coordinates']
            points.append((lon, lat))
            
            # Add classification 
            luc.append(feature['properties']['class'])
    return points, luc

In [7]:
# Function to transform unprojected coordinates to projected coordinates
def transform_coordinates(coordinates, epsg_s, epsg_t):
    source_crs = pyproj.CRS(f'EPSG:{epsg_s}') 
    target_crs = pyproj.CRS(f'EPSG:{epsg_t}')  
    transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)
    transformed_coords = [transformer.transform(lon, lat) for lon, lat in coordinates]
    
    transformed_coords_int = [[int(tc[0]), int(tc[1])] for tc in transformed_coords]
    return transformed_coords_int

In [8]:
# Function to extract pixel values from a GeoTIFF at given coordinates
def extract_pixel_values(b_g, transformed_coords):
    gt = b_g.GetGeoTransform()
    b_rst = b_g.GetRasterBand(1)
    
    values = []

    for lon, lat in transformed_coords:
        px = int((lon - gt[0]) / gt[1])  # Convert longitude to pixel x
        py = int((lat - gt[3]) / gt[5])  # Convert latitude to pixel y

        value = b_rst.ReadAsArray(px, py, 1, 1)[0][0]
        values.append(value)
    
    # Empty raster 
    b_rst = None
    
    return values

# Read ML STAC Item

In [9]:
# Define name of the ML item
item_label_fname = "item-label-train.json"

In [10]:
item = pystac.read_file(item_label_fname)
display(item.properties)

{'ml-aoi:split': 'train',
 'label:description': 'Land cover labels',
 'label:type': 'vector',
 'label:properties': ['class'],
 'label:classes': [{'NO_DATA': 0,
   'SATURATED_OR_DEFECTIVE': 1,
   'CAST_SHADOWS': 2,
   'CLOUD_SHADOWS': 3,
   'VEGETATION': 4,
   'NOT_VEGETATED': 5,
   'WATER': 6,
   'UNCLASSIFIED': 7,
   'CLOUD_MEDIUM_PROBABILITY': 8,
   'CLOUD_HIGH_PROBABILITY': 9,
   'THIN_CIRRUS': 10,
   'SNOW or ICE': 11}],
 'label:tasks': ['segmentation', 'regression'],
 'label:methods': ['manual'],
 'version': '0.1',
 'deprecated': False,
 'datetime': '2023-05-29T13:57:36.729558Z'}

## Load S2 scene

In [11]:
s2_href = [l.target for l in item.links if l.rel == 'source'][0]
print('href of the S2 scene:', s2_href)

href of the S2 scene: https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220524_0_L2A


In [12]:
# Read STAC Item
s2item = pystac.read_file(s2_href)
display(s2item.properties)

{'datetime': '2022-05-24T19:03:29Z',
 'platform': 'sentinel-2a',
 'constellation': 'sentinel-2',
 'instruments': ['msi'],
 'gsd': 10,
 'view:off_nadir': 0,
 'proj:epsg': 32610,
 'sentinel:utm_zone': 10,
 'sentinel:latitude_band': 'T',
 'sentinel:grid_square': 'FK',
 'sentinel:sequence': '0',
 'sentinel:product_id': 'S2A_MSIL2A_20220524T184921_N0400_R113_T10TFK_20220525T004817',
 'sentinel:data_coverage': 100,
 'eo:cloud_cover': 0.24,
 'sentinel:valid_cloud_cover': True,
 'sentinel:processing_baseline': '04.00',
 'sentinel:boa_offset_applied': True,
 'created': '2022-05-25T03:20:43.295Z',
 'updated': '2022-05-25T03:20:43.295Z'}

In [13]:
print(f'Available bands: {list(s2item.assets.keys())}')
epsg_t = s2item.properties['proj:epsg']
print(f'Target EPSG:{epsg_t}')

Available bands: ['thumbnail', 'overview', 'info', 'metadata', 'visual', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'AOT', 'WVP', 'SCL']
Target EPSG:32610


## Load geojson points

In [14]:
geojson_href = item.assets['labels'].href
print('href of the geojson file:', geojson_href)

href of the geojson file: label-train.geojson


In [15]:
# Open asset (geojson format) and read all the coordinates within 
coordinates, luc = read_geojson_coordinates(geojson_href)
coordinates[:5], luc[:5]

([(-121.27859163156747, 40.38218701036475),
  (-121.19953674120562, 39.91585786896506),
  (-121.33843034984275, 40.4972903822101),
  (-121.68743345874819, 40.07457719936124),
  (-121.26139230895694, 40.290492371933254)],
 [4, 4, 5, 4, 5])

In [16]:
# Transofrm coordinates
epsg_s = '4326'
transformed_coords = transform_coordinates(coordinates, epsg_s, epsg_t)
transformed_coords[:5]

[[646120, 4471599],
 [653880, 4419970],
 [640800, 4484280],
 [611920, 4436859],
 [647780, 4461450]]

## Extract values of selected band(s) for each point in the geojson file
Select a band or a list of bands, ie ```[`B02`, `B03`, `B04`, `B08`,`SCL`]```

In [17]:
bands = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B11', 'B12']

In [23]:
band = bands[0]
assert band in list(s2item.assets.keys()), f'Band {band} does not exist'

In [24]:
print('Band:', band)
b_metadata = s2item.assets[band].to_dict()
cbn = b_metadata['eo:bands'][0]['common_name']
print(f'- Common Band Name: {cbn}')
print(f'- Res: {b_metadata["gsd"]}m')
# Extract band
b_href = s2item.assets[band].href
print('- href:', b_href)

# Get gdal object
b_g = gdal.Open(b_href)

Band: B02
- Common Band Name: blue
- Res: 10m
- href: https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2022/5/S2A_10TFK_20220524_0_L2A/B02.tif


In [25]:
pixel_values = extract_pixel_values(b_g, transformed_coords)

# print(pixel_values)
i = 0
# Print the extracted pixel values
for (lon0, lat0), (lon1, lat1), value, lu in zip(coordinates, transformed_coords, pixel_values, luc):
    print(f"Coords (Unprj): {np.round(lon0,3)}, {np.round(lat0,3)} - Coords (Prj): {lon1}, {lat1} - Pixel Value: {value} - LUC: {lu}")
    i+=1 
    if i == 5: break

# Empty b_g
#b_g = None

Coords (Unprj): -121.279, 40.382 - Coords (Prj): 646120, 4471599 - Pixel Value: 363 - LUC: 4
Coords (Unprj): -121.2, 39.916 - Coords (Prj): 653880, 4419970 - Pixel Value: 448 - LUC: 4
Coords (Unprj): -121.338, 40.497 - Coords (Prj): 640800, 4484280 - Pixel Value: 432 - LUC: 5
Coords (Unprj): -121.687, 40.075 - Coords (Prj): 611920, 4436859 - Pixel Value: 312 - LUC: 4
Coords (Unprj): -121.261, 40.29 - Coords (Prj): 647780, 4461450 - Pixel Value: 394 - LUC: 5


## Make Pandas dataframe
**Note**: The *pandas* dataframe will be used as input for the EDA Notebook from the ARSET training.

In [26]:
# make dictionary
data = {'long': [x[0] for x in transformed_coords], 
        'lat': [x[1] for x in transformed_coords], 
        'LUC': luc,
        cbn: pixel_values,
       }
#data

In [27]:
if df is None: 
    print('Creating Dataframe')
    # Create a DataFrame from the dictionary
    df = pd.DataFrame(data)
    df.index.name = 'Index'
    
else: 
    print('Adding to existing Dataframe')
    
    # Create temp dataframe
    df2 = pd.DataFrame(data)
    df2.index.name = 'Index'
    
    # Assert the two dataframes have the same long and lat values
    assert df['long'].isin(df2['long']).value_counts().values[0] == len(pixel_values)
    assert df['lat'].isin(df2['lat']).value_counts().values[0] == len(pixel_values)
    
    # Merge temp dataframe with original dataframe, based on matching columns
    df = pd.merge(df, df2, on=['Index', 'long', 'lat', 'LUC'])   
    # Empty memory
    df2 = None
    
display(df)

Adding to existing Dataframe


,long,lat,LUC,nir,blue
Index,,,,,
0,646120,4471599,4,1115,363
1,653880,4419970,4,1062,448
2,640800,4484280,5,1534,432
3,611920,4436859,4,1952,312
4,647780,4461450,5,1454,394
...,...,...,...,...,...
395,668920,4432610,4,61,1
396,679400,4444339,5,2927,260
397,672460,4433990,4,1265,462


In [28]:
df# = df.drop(columns=['LUC_x','LUC_y'])

,long,lat,LUC,nir,blue
Index,,,,,
0,646120,4471599,4,1115,363
1,653880,4419970,4,1062,448
2,640800,4484280,5,1534,432
3,611920,4436859,4,1952,312
4,647780,4461450,5,1454,394
...,...,...,...,...,...
395,668920,4432610,4,61,1
396,679400,4444339,5,2927,260
397,672460,4433990,4,1265,462


In [29]:
# # Export dataframe 
df.to_csv('df_extractedpixels.csv')

In [30]:
print('END')

END
